# Shared Functions

In [ ]:
import os
import time
import logging
from datetime import datetime
from functools import reduce

import pandas as pd
from pyspark.sql import SparkSession, functions as F
from pyspark.sql.functions import (
    broadcast,
    col,
    coalesce,
    concat,
    current_timestamp,
    date_sub,
    first,
    lead,
    lit,
    lower,
    regexp_replace,
    sum as _sum,
    trim,
    when,
)
from pyspark.sql.types import DateType, DecimalType
from pyspark.sql.window import Window


# Function to retrieve the Bronze storage path dynamically based on the current workspace
def get_se_bronze_path():
    import sempy.fabric as fabric
    curr_workspace = fabric.get_notebook_workspace_id()  # Get the current workspace ID
    workspaces = fabric.list_workspaces()  # List all available workspaces
    curr_workspace_item = workspaces[workspaces.Id == curr_workspace]  # Find current workspace details
    lakehouse_storage = curr_workspace_item['Name'].iloc[0]  # Get the workspace name
    
    # Mapping of workspaces to their respective Bronze lakehouse paths
    workspace_lakehouse_dict = {
        'DevSecure_Enclave': 'abfss://DevSecure_Enclave@onelake.dfs.fabric.microsoft.com/V6_SE_Bronze.Lakehouse/Tables',
        'Secure_Enclave': 'abfss://Secure_Enclave@onelake.dfs.fabric.microsoft.com/V6_SE_Bronze.Lakehouse/Tables'
    }
    
    return workspace_lakehouse_dict.get(lakehouse_storage, None)  # Return the appropriate path or None if not found

def get_se_pewter_path():
    import sempy.fabric as fabric
    curr_workspace = fabric.get_notebook_workspace_id()  # Get the current workspace ID
    workspaces = fabric.list_workspaces()  # List all available workspaces
    curr_workspace_item = workspaces[workspaces.Id == curr_workspace]  # Find current workspace details
    lakehouse_storage = curr_workspace_item['Name'].iloc[0]  # Get the workspace name
    
    # Mapping of workspaces to their respective Bronze lakehouse paths
    workspace_lakehouse_dict = {
        'DevSecure_Enclave': 'abfss://DevSecure_Enclave@onelake.dfs.fabric.microsoft.com/V6_SE_Pewter.Lakehouse/Tables',
        'Secure_Enclave': 'abfss://Secure_Enclave@onelake.dfs.fabric.microsoft.com/V6_SE_Pewter.Lakehouse/Tables'
    }
    

    return workspace_lakehouse_dict.get(lakehouse_storage, None)  # Return the appropriate path or None if not found

# Function to retrieve the Silver storage path dynamically based on the current workspace
def get_se_silver_path():
    import sempy.fabric as fabric
    curr_workspace = fabric.get_notebook_workspace_id()  # Get the current workspace ID
    workspaces = fabric.list_workspaces()  # List all available workspaces
    curr_workspace_item = workspaces[workspaces.Id == curr_workspace]  # Find current workspace details
    lakehouse_storage = curr_workspace_item['Name'].iloc[0]  # Get the workspace name

    workspace_lakehouse_dict = {
        'DevSecure_Enclave': 'abfss://DevSecure_Enclave@onelake.dfs.fabric.microsoft.com/V6_SE_Silver.Lakehouse/Tables',
        'Secure_Enclave': 'abfss://Secure_Enclave@onelake.dfs.fabric.microsoft.com/V6_SE_Silver.Lakehouse/Tables'
        }
    return workspace_lakehouse_dict.get(lakehouse_storage, None)  # Return the appropriate path or None if not found


def get_se_gold_path():
    import sempy.fabric as fabric
    curr_workspace = fabric.get_notebook_workspace_id()  # Get the current workspace ID
    workspaces = fabric.list_workspaces()  # List all available workspaces
    curr_workspace_item = workspaces[workspaces.Id == curr_workspace]  # Find current workspace details
    lakehouse_storage = curr_workspace_item['Name'].iloc[0]  # Get the workspace name
    
    # Mapping of workspaces to their respective Silver lakehouse paths
    workspace_lakehouse_dict = {
        'DevSecure_Enclave': 'abfss://DevSecure_Enclave@onelake.dfs.fabric.microsoft.com/V6_SE_Gold.Lakehouse/Tables', 
        'Secure_Enclave': 'abfss://Secure_Enclave@onelake.dfs.fabric.microsoft.com/V6_SE_Gold.Lakehouse/Tables'
    }

    return workspace_lakehouse_dict.get(lakehouse_storage, None)  # Return the appropriate path or None if not found


# Function to retrieve the Bronze storage path dynamically based on the current workspace
def get_bronze_path():
    import sempy.fabric as fabric
    curr_workspace = fabric.get_notebook_workspace_id()  # Get the current workspace ID
    workspaces = fabric.list_workspaces()  # List all available workspaces
    curr_workspace_item = workspaces[workspaces.Id == curr_workspace]  # Find current workspace details
    lakehouse_storage = curr_workspace_item['Name'].iloc[0]  # Get the workspace name
    
    # Mapping of workspaces to their respective Bronze lakehouse paths
    workspace_lakehouse_dict = {
        'DevSecure_Enclave': 'abfss://DevBronze@onelake.dfs.fabric.microsoft.com/BronzeLake.Lakehouse/Tables',
        'Secure_Enclave': 'abfss://ProdBronze@onelake.dfs.fabric.microsoft.com/BronzeLake.Lakehouse/Tables'
    }
    

    return workspace_lakehouse_dict.get(lakehouse_storage, None)  # Return the appropriate path or None if not found

def get_pewter_path():
    import sempy.fabric as fabric
    curr_workspace = fabric.get_notebook_workspace_id()  # Get the current workspace ID
    workspaces = fabric.list_workspaces()  # List all available workspaces
    curr_workspace_item = workspaces[workspaces.Id == curr_workspace]  # Find current workspace details
    lakehouse_storage = curr_workspace_item['Name'].iloc[0]  # Get the workspace name
    
    # Mapping of workspaces to their respective Bronze lakehouse paths
    workspace_lakehouse_dict = {
            'DevSecure_Enclave': 'abfss://DevPewter@onelake.dfs.fabric.microsoft.com/PewterPreProcessed.Lakehouse/Tables', 
            'Secure_Enclave': 'abfss://ProdPewter@onelake.dfs.fabric.microsoft.com/PewterPreProcessed.Lakehouse/Tables',
    }
    

    return workspace_lakehouse_dict.get(lakehouse_storage, None)  # Return the appropriate path or None if not found

# Function to retrieve the Silver storage path dynamically based on the current workspace
def get_silver_path():
    import sempy.fabric as fabric
    curr_workspace = fabric.get_notebook_workspace_id()  # Get the current workspace ID
    workspaces = fabric.list_workspaces()  # List all available workspaces
    curr_workspace_item = workspaces[workspaces.Id == curr_workspace]  # Find current workspace details
    lakehouse_storage = curr_workspace_item['Name'].iloc[0]  # Get the workspace name
    
    workspace_lakehouse_dict = {
            'DevSecure_Enclave': 'abfss://DevSilver@onelake.dfs.fabric.microsoft.com/SilverTransformed.Lakehouse/Tables', 
            'Secure_Enclave': 'abfss://ProdSilver@onelake.dfs.fabric.microsoft.com/SilverTransformed.Lakehouse/Tables'
        }
    
    return workspace_lakehouse_dict.get(lakehouse_storage, None)  # Return the appropriate path or None if not found


def get_gold_path():
    import sempy.fabric as fabric
    curr_workspace = fabric.get_notebook_workspace_id()  # Get the current workspace ID
    workspaces = fabric.list_workspaces()  # List all available workspaces
    curr_workspace_item = workspaces[workspaces.Id == curr_workspace]  # Find current workspace details
    lakehouse_storage = curr_workspace_item['Name'].iloc[0]  # Get the workspace name
    
    # Mapping of workspaces to their respective Silver lakehouse paths
    workspace_lakehouse_dict = {
            'DevSecure_Enclave': 'abfss://DevGold@onelake.dfs.fabric.microsoft.com/DevGold_Lake.Lakehouse/Tables', 
            'Secure_Enclave': 'abfss://34c8d809-ab93-4e32-acb3-911afc62b7f9@onelake.dfs.fabric.microsoft.com/7a6791f6-1a48-43e1-80f7-019e4be82bcc/Tables'
    }

    return workspace_lakehouse_dict.get(lakehouse_storage, None)  # Return the appropriate path or None if not found


# Function to save a DataFrame as a Delta table in either the Silver or Bronze layer
def save_table(df, transformed_table_name, layer):
    spark.conf.set('spark.sql.caseSensitive', True)  # Ensure case sensitivity in column names
    
    # Determine destination storage path based on the specified layer
    if layer.lower() == "silver":
        destination_base_path = get_silver_path()
    elif layer.lower() == "bronze":
        destination_base_path = get_bronze_path()
    elif layer.lower() == "pewter":
        destination_base_path = get_pewter_path()
    else:
        raise ValueError("Invalid layer specified. Use 'bronze', 'pewter', or 'silver'.")

    # If the storage path cannot be determined, raise an error
    if destination_base_path is None:
        raise RuntimeError(f"Could not determine the {layer} storage path.")

    # Construct the full destination path
    destination_path = f"{destination_base_path}/{transformed_table_name}"

    # Save the DataFrame as a Delta table
    df.write.format("delta") \
        .mode("overwrite") \
        .option("overwriteSchema", "true") \
        .save(destination_path)

    print(f"Table '{transformed_table_name}' saved to {destination_path}")

# Function to retrieve table data using Spark SQL
def get_table_data_sql(table_name, selected_columns=None, database="dbo"):
    table = f"{database}.{table_name}"  # Define the fully qualified table name
    columns_str = ", ".join(selected_columns) if selected_columns else "*"  # Select specified columns or all columns
    query = f"SELECT {columns_str} FROM {table}"  # Construct the SQL query
    
    return spark.sql(query)  # Execute the query and return the result as a DataFrame

# Function to retrieve table data using Spark's read.load() instead of SQL
def get_table_data(table_name, lakehouse_path, selected_columns=None):
    table_path = f"{lakehouse_path}/{table_name}"  # Construct the table path
    df = spark.read.format('delta').load(table_path)  # Read the data from storage

    # Select only the specified columns if provided
    if selected_columns:
        df = df.select(*selected_columns)
    
    return df  # Return the DataFrame

# Function to setup logger capabilities
def setup_logger(log_name):
    """
    Sets up a logger with both console and file handlers.

    :param log_name: Name of the log (used for log directory and filename)
    :return: Configured logger instance
    """
    log_path = "/lakehouse/default/Files/Logs"
    logging_path = os.path.join(log_path, log_name)

    # Ensure the log directory exists
    os.makedirs(logging_path, exist_ok=True)

    # Create log file name with timestamp
    log_filename = f"{log_name}_{datetime.now().strftime('%Y%m%d_%H%M%S')}.log"
    log_file_path = os.path.join(logging_path, log_filename)

    # Create a logger with the given name
    logger = logging.getLogger(log_name)
    logger.setLevel(logging.INFO)

    # Remove existing handlers to avoid duplicates
    if logger.hasHandlers():
        logger.handlers.clear()

    # Console handler
    stream_handler = logging.StreamHandler()
    stream_handler.setLevel(logging.INFO)
    stream_formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
    stream_handler.setFormatter(stream_formatter)
    logger.addHandler(stream_handler)

    # File handler
    file_handler = logging.FileHandler(log_file_path, mode='a')
    file_handler.setLevel(logging.INFO)
    file_formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
    file_handler.setFormatter(file_formatter)
    logger.addHandler(file_handler)

    logger.info(f"{log_name} logging setup complete.")

    return logger  # Return the logger for direct use

# Function to move tables based on prefix to Gold Lakehouse - dynamically adjusts on Dev and Prod
def move_tables_to_gold(
    prefix,
):
    """
    Transfers tables from a Silver (transformed) path to a Gold path if their names start with a given prefix.

    Parameters:
    - silver_path (str): Source path where transformed Delta tables are stored.
    - gold_path (str): Destination path for Gold layer Delta tables.
    - prefix (str): Prefix used to identify which tables to move.
    - spark_session (SparkSession, optional): The Spark session to use.
    - logger (Logger, optional): Logger instance to capture logs.
    """
    silver_path = get_silver_path()
    gold_path = get_gold_path()

    if logger:
        logger.info(f'Starting transfer of tables with prefix "{prefix}" from Silver ({silver_path}) to Gold ({gold_path}).')

    tables_df = mssparkutils.fs.ls(silver_path)
    table_names = [entry.name for entry in tables_df if entry.name.startswith(prefix)]

    for table in table_names:
        try:
            df = spark.read.format("delta").load(f"{silver_path}/{table}")
            df.write.format("delta").mode("overwrite").option("overwriteSchema", "true").save(f"{gold_path}/{table}")
            
            if logger:
                logger.info(f"Successfully moved {table} to {gold_path}")
        except Exception as e:
            if logger:
                logger.error(f"Failed to move {table} to Gold: {str(e)}")
            else:
                print(f"Failed to move {table}: {e}")



### Guardrail Functions

In [ ]:
def validate_row_count(denorm_df, fact_df):
    fact_count = fact_df.count()
    denorm_count = denorm_df.count()

    if denorm_count != fact_count:
        raise ValueError(f"❌ Row count validation check failed: fact df has {fact_count} rows, while denorm_df has {denorm_count} rows.")

    print(f"✅ Row count validation check passed: {fact_count} rows.")
    return True

def validate_duplicates(denorm_df, columns):
    duplicate_count = denorm_df.groupBy(columns).count().filter("count > 1").count()

    if duplicate_count > 0:
        raise ValueError(f"❌ Duplicate check failed: found {duplicate_count} duplicate rows based on columns {columns}.")

    print(f"✅ Duplicates validation check passed for columns: {columns}.")
    return True

def validate_nulls(df, columns):
    if isinstance(columns, str):
        columns = [columns]

    null_counts = df.select([
        F.count(F.when(F.col(col).isNull(), col)).alias(col) for col in columns
    ]).collect()[0].asDict()

    failing_columns = {col: count for col, count in null_counts.items() if count > 0}

    if failing_columns:
        raise ValueError(f"❌ Null check failed: found nulls in columns: {failing_columns}")

    print(f"✅ Nulls validation check passed for columns: {columns}.")
    return True


### Function to display info on all the functions inside SharedFunctions Notebook

In [ ]:
def shared_function_details():
    functions_info = {
        "get_bronze_path": {
            "purpose": "Retrieves the Bronze storage path dynamically based on the current workspace.",
            "parameters": "None",
            "returns": "The Bronze storage path as a string or None if not found."
        },
        "get_silver_path": {
            "purpose": "Retrieves the Silver storage path dynamically based on the current workspace.",
            "parameters": "type (str): 'raw' for raw Silver data, any other value for transformed Silver data.",
            "returns": "The Silver storage path as a string or None if not found."
        },
        "save_table": {
            "purpose": "Saves a Spark DataFrame as a Delta table in either the Silver or Bronze layer.",
            "parameters": """
                df (DataFrame): The Spark DataFrame to save.
                transformed_table_name (str): The name of the table.
                layer (str): The storage layer ('silver' or 'bronze').""",
            "returns": "None (Saves the DataFrame and prints confirmation)."
        },
        "get_table_data_sql": {
            "purpose": "Retrieves table data using Spark SQL.",
            "parameters": """
                table_name (str): The name of the table.
                selected_columns (list, optional): A list of column names to select. Defaults to None (selects all columns).
                database (str, optional): The database schema. Defaults to 'dbo'.""",
            "returns": "A Spark DataFrame containing the queried data."
        },
        "get_table_data": {
            "purpose": "Retrieves table data using Spark's read.load() instead of SQL.",
            "parameters": """
                table_name (str): The name of the table.
                lakehouse_path (str): The base path of the lakehouse.
                type (str, optional): Specifies whether to get 'raw' or transformed Silver data. Defaults to 'raw'.
                selected_columns (list, optional): A list of columns to select. Defaults to None (selects all columns).""",
            "returns": "A Spark DataFrame containing the retrieved data."
        }
    }

    for func_name, details in functions_info.items():
        print(f"\nFunction: {func_name}")
        print(f"  Purpose: {details['purpose']}")
        print(f"  Parameters: {details['parameters']}")
        print(f"  Returns: {details['returns']}")

In [ ]:
import json
import time
from time import sleep
from datetime import datetime
from typing import Optional, Tuple, Any, Dict

import pandas as pd
from tqdm import tqdm
import sempy.fabric as fabric


# ----------------------------
# Helpers
# ----------------------------

def is_dev_workspace() -> bool:
    workspace_id = fabric.get_workspace_id()
    workspace_name = fabric.resolve_workspace_name(workspace_id)
    return "dev" in (workspace_name or "").lower()


def _safe_get(d: Dict[str, Any], *paths, default=None):
    """
    Safely traverse nested dicts/lists to extract a value.
    Example: _safe_get(obj, 'a', 0, 'b') -> obj['a'][0]['b'] if present, else default.
    """
    cur = d
    try:
        for p in paths:
            if isinstance(cur, list) and isinstance(p, int):
                cur = cur[p]
            elif isinstance(cur, dict):
                cur = cur.get(p)
            else:
                return default
        return cur if cur is not None else default
    except Exception:
        return default


def _extract_failure_info(payload: Dict[str, Any]) -> Tuple[Optional[str], Optional[str], Optional[str]]:
    """
    Try a variety of common keys/paths that might contain failure reason/message/details.
    Returns (failure_reason, error_message, details_blob)
    """
    candidate_reason_keys = [
        "failureReason", "Failure Reason", "reason", "statusReason", "cancellationReason"
    ]
    candidate_message_keys = [
        "errorMessage", "Error Message", "message", "statusMessage", "failureMessage", "exceptionMessage", "detail"
    ]

    # Flat lookups
    reason = None
    for k in candidate_reason_keys:
        v = payload.get(k)
        if v:
            reason = v
            break

    message = None
    for k in candidate_message_keys:
        v = payload.get(k)
        if v:
            message = v
            break

    # Nested common shapes
    if not reason:
        reason = _safe_get(payload, "error", "code")
    if not message:
        message = _safe_get(payload, "error", "message") or _safe_get(payload, "statusDetails", "message")

    # Some services include arrays of errors
    if not message:
        errs = _safe_get(payload, "errors") or _safe_get(payload, "error", "details")
        if isinstance(errs, list) and errs:
            message = errs[0].get("message") or errs[0].get("detail") or str(errs[0])

    # Compact details blob (pretty JSON) so you have raw material for debugging
    try:
        details_blob = json.dumps(payload, indent=2, ensure_ascii=False)
    except Exception:
        details_blob = None

    return reason, message, details_blob


def format_datetime(dt):
    if pd.isna(dt):
        return "N/A"
    if isinstance(dt, str):
        try:
            return datetime.fromisoformat(dt).strftime('%Y-%m-%d %H:%M:%S')
        except ValueError:
            return dt
    if isinstance(dt, datetime):
        return dt.strftime('%Y-%m-%d %H:%M:%S')
    return str(dt)


# ----------------------------
# Fabric Job Runner (Pipelines & Notebooks)
# ----------------------------

def _run_fabric_job(
    artifact_name: str,
    job_type: str,
    workspace_name: str = None,
    poll_interval: int = 10
) -> Optional[str]:
    """
    Triggers and polls a Fabric job (Pipeline or RunNotebook).
    On failure, attempts to print failure reason and message from the job instance payload.
    Returns job_id on success, else None.
    """
    client = fabric.FabricRestClient()

    if workspace_name:
        workspace_id = fabric.resolve_workspace_id(workspace_name)
    else:
        workspace_id = fabric.get_workspace_id()

    item_id = fabric.resolve_item_id(artifact_name, workspace=workspace_id)

    supported_job_types = {"Pipeline", "RunNotebook"}
    if job_type not in supported_job_types:
        print(f"Unsupported job type: {job_type}. Supported types are: {supported_job_types}")
        return None

    url = f"/v1/workspaces/{workspace_id}/items/{item_id}/jobs/instances?jobType={job_type}"

    # Start job
    if job_type == "RunNotebook":
        response = client.post(url, json={"executionData": {}})
    else:
        response = client.post(url)

    if response.status_code != 202:
        print(f"{job_type} '{artifact_name}' failed to start. "
              f"Status code: {response.status_code}, {response.text}")
        return None

    job_id = response.headers["Location"].split("/")[-1]
    print(f"🔄 {job_type} '{artifact_name}' triggered. Job ID: {job_id}")

    status_url = f"/v1/workspaces/{workspace_id}/items/{item_id}/jobs/instances/{job_id}"
    last_status = None
    start_ts = time.time()

    time.sleep(poll_interval)

    while True:
        status_response = client.get(status_url)

        if status_response.status_code != 200:
            print(f"⚠️ Unexpected status code while polling: {status_response.status_code}")
            time.sleep(poll_interval)
            continue

        # Parse payload
        try:
            status_data = status_response.json()
            if isinstance(status_data, str):
                status_data = json.loads(status_data)
        except Exception as e:
            print(f"Failed to parse job status: {e}")
            print(f"Raw: {status_response.text}")
            time.sleep(poll_interval)
            continue

        status = status_data.get("status", "Unknown")
        if status != last_status:
            print(f"⏱️ Job Status: {status}")
            last_status = status

        if status == "Completed":
            elapsed = round(time.time() - start_ts, 2)
            print(f"✅ {job_type} '{artifact_name}' completed in {elapsed} seconds.")
            return job_id

        if status in ("Failed", "Cancelled"):
            elapsed = round(time.time() - start_ts, 2)
            print(f"❌ {job_type} '{artifact_name}' did not complete. Status: {status}")
            # Try to extract reason/message
            reason, message, details_blob = _extract_failure_info(status_data)

            # Sometimes there are run-level logs/URIs
            log_uri = (
                _safe_get(status_data, "logUri")
                or _safe_get(status_data, "logsUri")
                or _safe_get(status_data, "output", "logUri")
            )

            print("   ➡️ Failure Reason:", reason or "N/A")
            print("   ➡️ Error Message :", message or "N/A")
            if log_uri:
                print("   📄 Logs URI      :", log_uri)
            print(f"   ⏱️ Duration      : {elapsed} seconds.")

            # If nothing specific was found, dump a trimmed details blob (last resort)
            if not (reason or message) and details_blob:
                # Keep it compact to avoid overwhelming logs
                preview = details_blob[:2000] + (" …(truncated)…" if len(details_blob) > 2000 else "")
                print("   🔎 Raw Payload (preview):")
                print(preview)
            return None

        time.sleep(poll_interval)


def run_notebook(artifact_name: str, workspace_name: Optional[str] = None, poll_interval: int = 10) -> Optional[str]:
    return _run_fabric_job(
        artifact_name, job_type="RunNotebook", workspace_name=workspace_name, poll_interval=poll_interval
    )


def run_pipeline(artifact_name: str, workspace_name: Optional[str] = None, poll_interval: int = 10) -> Optional[str]:
    return _run_fabric_job(
        artifact_name, job_type="Pipeline", workspace_name=workspace_name, poll_interval=poll_interval
    )


# ----------------------------
# Dataset Refresh Runner
# ----------------------------

def refresh_dataset(dataset: str, workspace: str, poll_interval: int = 10) -> Optional[str]:
    """
    Triggers and polls a dataset refresh. Prints detailed failure info when status != Completed.
    Returns the Refresh Request ID (if obtainable) on success, else None.
    """
    print(f"🔄 Starting dataset refresh: '{dataset}' in workspace '{workspace}'")

    try:
        fabric.refresh_dataset(dataset, workspace)
    except Exception as e:
        print(f"❌ Failed to trigger dataset refresh: {e}")
        return None

    last_status = None
    start_wall = time.time()

    time.sleep(poll_interval)

    refresh_request_id = None

    while True:
        try:
            df = fabric.list_refresh_requests(dataset=dataset, workspace=workspace)
            if df is None or len(df.index) == 0:
                raise RuntimeError("No refresh requests found.")
            latest_request = df.iloc[0]
        except Exception as e:
            print(f"⚠️ Failed to fetch refresh request: {e}")
            time.sleep(poll_interval)
            continue

        status = latest_request.get('Status', 'Unknown')
        start = latest_request.get('Start Time')
        end = latest_request.get('End Time')

        # Capture request ID if present
        refresh_request_id = latest_request.get('Id') or latest_request.get('ID') or refresh_request_id

        if status != last_status:
            print(f"⏱️ Refresh Status: {status} | Start: {format_datetime(start)} | End: {format_datetime(end)}")
            last_status = status

        if status in ['Completed', 'Failed', 'Cancelled']:
            try:
                elapsed = round((end - start).total_seconds(), 2)
            except Exception:
                elapsed = round(time.time() - start_wall, 2)

            if status == 'Completed':
                print(f"✅ Dataset '{dataset}' refresh completed in {elapsed} seconds.")
                if refresh_request_id:
                    print(f"   🆔 Refresh Request ID: {refresh_request_id}")
                return refresh_request_id

            # Extract failure columns if available
            failure_reason = latest_request.get('Failure Reason') or latest_request.get('Reason') or None
            error_message = latest_request.get('Error Message') or latest_request.get('Message') or None

            # Build a dict for a raw row preview if we don't have clear fields
            row_dict = {}
            try:
                row_dict = latest_request.to_dict()
            except Exception:
                pass

            print(f"❌ Dataset '{dataset}' refresh did not complete successfully.")
            print(f"   ➡️ Status         : {status}")
            print(f"   ➡️ Failure Reason : {failure_reason or 'N/A'}")
            print(f"   ➡️ Error Message  : {error_message or 'N/A'}")
            if refresh_request_id:
                print(f"   🆔 Refresh Request ID: {refresh_request_id}")
            print(f"   ⏱️ Duration       : {elapsed} seconds.")

            # If neither failure_reason nor error_message present, dump a compact preview of the row
            if not (failure_reason or error_message) and row_dict:
                try:
                    blob = json.dumps(row_dict, indent=2, ensure_ascii=False)
                    preview = blob[:2000] + (" …(truncated)…" if len(blob) > 2000 else "")
                    print("   🔎 Latest Request Row (preview):")
                    print(preview)
                except Exception:
                    pass

            return None

        time.sleep(poll_interval)


# ----------------------------
# Example usage (uncomment to run)
# ----------------------------

# if __name__ == "__main__":
#     # Pipelines / Notebooks
#     run_pipeline("My Pipeline", workspace_name="My Workspace", poll_interval=10)
#     run_notebook("My Notebook", workspace_name="My Workspace", poll_interval=10)
#
#     # Dataset refresh
#     refresh_dataset("My Dataset", workspace="My Workspace", poll_interval=10)


In [ ]:
def get_gold_workspace():
    import sempy.fabric as fabric
    curr_workspace = fabric.get_notebook_workspace_id()  # Get the current workspace ID
    workspaces = fabric.list_workspaces()  # List all available workspaces
    curr_workspace_item = workspaces[workspaces.Id == curr_workspace]  # Find current workspace details
    lakehouse_storage = curr_workspace_item['Name'].iloc[0]  # Get the workspace name
    
    # Mapping of workspaces to their respective Silver lakehouse paths
    workspace_lakehouse_dict = {
            'DevSecure_Enclave': 'DevGold', 
            'Secure_Enclave': 'Data and Analytics'
    }

    return workspace_lakehouse_dict.get(lakehouse_storage, None)  # Return the appropriate path or None if not found